In [ ]:
# Check multithreading config:
Base.Threads.nthreads()

In [ ]:
# Check active package versions:
using Pkg; pkg"status"

<h1 style="text-align: center;">
    <span style="display: block; text-align: center;">
        Introduction to
    </span>
    <span style="display: block; text-align: center;">
        <img alt="Julia" src="images/logos/julia-logo.svg" style="height: 2em; display: inline-block; margin: 1em;"/>
    </span>
    <span style="display: block; text-align: center;">
        Part 1
    </span>
</h1>

<div style="text-align: center;">
    <div style="text-align: center; display: inline-block; vertical-align: middle;">
        Ludger Pähler <br/>
        <small>
        TUM, Chair of Aerodynamics and Fluid mechanics <br/>
            <a href="mailto:ludger.paehler@tum.de" target="_blank">ludger.paehler@tum.de</a>
        </small>
    </div>
    <div style="text-align: center; display: inline-block;">
        <img src="images/logos/tum-logo.svg" style="height: 8em; display: inline-block;  vertical-align: middle; margin: 1em;"/>
    </div>
</div>

<div style="text-align: center;">
    <p style="text-align: center; display: inline-block; vertical-align: middle;">
        Oliver Schulz<br>
        <small>
            Max Planck Institute for Physics <br/>
            <a href="mailto:oschulz@mpp.mpg.de" target="_blank">oschulz@mpp.mpg.de</a>
        </small>
    </p>
    <p style="text-align: center; display: inline-block; vertical-align: middle;">
        <img src="images/logos/mpg-logo.svg" style="height: 5em; display: inline-block; vertical-align: middle; margin: 1em;"/>
        <img src="images/logos/mpp-logo.svg" style="height: 5em; display: inline-block; vertical-align: middle; margin: 1em;"/>
    </p>
</div>

<p style="text-align: center;">
    Leibniz Supercomputing Centre - LRZ, Jan. 27th 2019
</p>

## Introduction

### Science needs code - but how to write it?

* Choice of programming language(s) matter!

* Need to balance:
    * Learning time
    * Productivity
    * Performance

* Usually involves compromises

### Programming Language Options

* C++:
    * Pro: Very fast (in expert hands)
    * Con: Complex, takes long time to master
    * Con: Straightforward tasks often result in lengthy code
    * Con: No memory management (General protection faults)  
    * No universal package management

### Programming Language Options

* Python:
    * Pro: Broad user base, popular first programming language
    * Pro: Easy to learn, good standard library
    * Con: Can't write time-critical loops in Python,  
      workarounds like Numba/Cython have many limitations
    * Con: Language itself fairly primitive, not very expressive
    * Con: Duck-Typing necessitates lots of test code
    * Con: No effective multi-threading

### What else is there?

* Fortran:
    * Pro: Math is really fast
    * Con: Old language, few modern concepts, shrinking user base
    * Do you *really* want to ...?


* Scala, Go, Kotlin etc.:
    * Pro: Lot's of individual strenghts
    * Con: Math either fast *or* generic *or* or complicated
    * Con: Calling C, Fortran or Phython code often difficult

### The 97 and the 3 Percent

> We should forget about small efficiencies, say about 97% of the time: *premature optimization is the root of all evil. Yet we should not pass up our opportunities in that critical 3%*.

Donald E. Knuth

* Some programming languages (e.g. Python) great for the 97% -  
  but can't make the 3% fast.
* Some other languages (e.g. C/C++, Fortran) can handle the 3% -  
  but makes the 97% complicated.

### The Two-language Problem

* Common approach nowadays:  
  Write time critical code in C/C++, rest in Python

* Pro: End-user can code comfortably in Python, with good performance

* Con: Complexity of C/C++ **plus** complexity of Python

* Con: Need proficiency in **two** languages, many users blocked  
  from contributing to important parts of code

* Con: Limits generic implementation of algorithms

* Con: Severely limits metaprogramming

### We were looking for a language ...
* as fast as C/C++
* as easy to learn and productive as Python
* with first class math support (vectors, matrices, etc.)
* with true functional programming
* with great Fortran/C/C++/Python integration
* with true metaprogramming (like Lisp or Scala)
* good at parallel and distributed programming
* suitable for for interactive, small and large applications

### Julia
* Designed for scientific/technical computing
* Originated at MIT, first public version 2012
* Covers the whole wish-list
* Clear focus on user productivity and software quality
* Rapid growth of user base and software packages
* Software package management:  
  Trivial to create and install packages
* Excellent REPL (console)
* Easy to call Fortran, C/C++ and Python code
* Current version (almost): Julia v1.3(-RC4)

### Julia Language Properties
* Fast: JAOT compilation to native CPU and GPU code
* Multiple-dispatch (more powerful than object-oriented):  
  solves the expression problem
* Dynamically typed, but types are fist-class
* Functional programming and metaprogramming
* First-class math support (like Fortran or Matlab)
* Local and distributed code execution
* State-of-the-art multi-threading: parallel code  
  can call parallel code that can call parallel code, ...,  
  without oversubscribing threads

### Hello World (and more) in Julia

In [ ]:
println("Hello, World!")

Let's define a function

In [ ]:
f(x, y) = x * y
f(20, 2.1)

Multiplication is also defined for vectors, so this works, too:

In [ ]:
f(4.2, [1, 2, 3, 4])

### Let's Look Under the Hood

In [ ]:
@code_llvm debuginfo=:none f(20, 2.1)

In [ ]:
@code_native debuginfo=:none f(20, 2.1)

### Multiple Dispatch

In [ ]:
foo(x::Integer, y::Number) = x * y
foo(x::Integer, y::AbstractString) = join(fill(y, x))

In [ ]:
foo(3, 4)

In [ ]:
foo(3, "abc")

In [ ]:
foo(4.5, 3)

In [ ]:
foo("a", "b")

### Functional Programming

In [ ]:
A = rand(10)
idxs = findall(x -> 0.2 < x < 0.6, A)

In [ ]:
A[idxs]

Even types are first-class values:

In [ ]:
mytype = Number

In [ ]:
subtypes(mytype)

Julia type hierarchy extends all the way down to primitive types:

In [ ]:
Float64 <: AbstractFloat <: Real <: Number <: Any

This is efficient (not runtime reflection):

In [ ]:
half_dynrange(T::Type{<:Number}) = (Int(typemax(T)) - Int(typemin(T))) / 2
half_dynrange(Int16)

In [ ]:
@code_llvm half_dynrange(Int16)

### Broadcasting

In [ ]:
A = [1.1, 2.2, 3.3]
B = [4.4, 5.5, 6.6]
broadcast((x, y) -> (x + y)^2, A, B)

Shorter broadcast syntax:

In [ ]:
(A .+ B) .^ 2

#### Loop Fusion and SIMD Vectorization

In [ ]:
foo(X, Y) = (X .+ Y) .^ 2
@code_llvm debuginfo=:none foo(A, B)

In [ ]:
@code_native foo(A, B)

### Let's Make a Plot

In [ ]:
using Plots
range = -π:0.01:π
plot(range, sin.(range) + rand(length(range)))

### Histograms are easy, too

In [ ]:
using Distributions
dist = Normal(0.0, 5.0)

In [ ]:
stephist(rand(dist, 10000))

### Python integration via PyCall.jl

In [ ]:
using PyCall

In [ ]:
numpy = pyimport("numpy")
numpy.zeros(5)

In [ ]:
A = rand(5)

In [ ]:
py"""type($A)"""

### C++ Integration via Cxx.jl

In [ ]:
using Cxx
cxxinclude("iostream")

message = "Hello, World"
out_stream = icxx" std::cout << $message << std::endl; "

In [ ]:
message = "Hello, World!"
l = signed(icxx""" std::string($message).size(); """)

In [ ]:
icxx""" $out_stream << "Message is " << $l << " characters long." << std::endl; 0; """;

### Running Julia code on GPUs

In [ ]:
ENV["JULIA_DEBUG"] = "CUDAnative"

using GPUArrays, CuArrays, CUDAnative, FFTW

f(a, b) = a^2 + b^2

A_cpu = rand(10^6); B_cpu = rand(10^6);
result_cpu = sum(f.(A_cpu, B_cpu))

A_gpu = CuArray(A_cpu); B_gpu = CuArray(B_cpu);
result_gpu = sum(f.(A_gpu, B_gpu))

result_cpu ≈ result_gpu

Can also write CUDA kernels in Julia by hand via [CUDAnative.jl](https://github.com/JuliaGPU/CUDAnative.jl).